# Concurrency and races

Below is an example of code with data races. Data Races occur when there is at least one read and another write at the same location. Consider the following example

In [ ]:
func dataRaceExample() {
    var count = 0
    
    func inc() {
        if count == 0 {
            count++
        }
    }
    
    func main() {
        go inc()
        go inc()
        time.Sleep(1 * time.Millisecond)
    }
    main()
}

dataRaceExample()


Data race is pretty obvious at `count++` since multiple user-threads read and write to it simultaenously
An improved version may look like this. Look under `concurrency/` directory for `go` code

In [ ]:
import (
	"context"
	"fmt"
	"sync"
	"time"
)

var (
	count = 0
	wg    sync.WaitGroup
	m     = sync.Mutex{}
)

func incrementCount(ctx context.Context, who string) {
	defer wg.Done()

	inc := 0
	for {
		select {
		case <-ctx.Done():
			fmt.Println(who, inc, " ... exiting")
			return
		case <-time.After(400 * time.Millisecond):
			inc += 1
			m.Lock()    // <<< sync access to data
			fmt.Println(who, inc, ":   now  :", count)
			count += 1
			fmt.Println(who, inc, ":   inc  :", count)
			m.Unlock()  // <<< end of sync block
		}
	}
}

func cancel() {
	fmt.Println(" .... cancel")
}

func main() {

	ctx, cancel := context.WithCancel(context.Background())

	wg.Add(3)
	go incrementCount(ctx, "A")
	go incrementCount(ctx, "                     B")
	go incrementCount(ctx, "                                           C")

	time.AfterFunc(3*time.Second, cancel)
	wg.Wait()

	fmt.Println("  ... end: ", count)   
}

main()

The program above solves the data race and by synchronising access to `count` using mutexes. Also note how `WaitGroup` is used to wait for the `goroutines` to be done.
The way this works is that `cancel` is called after 3 seconds then the timer (`func main:` `time.AfterFunc(..)`) times out. This send a signal on `ctx.Done()` and each go routine exits which results in a deferred call to `wg.Done()`. `wg.Wait()` ensures that all go-routines' `Done` is called before continuing to print `count`